In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("hwk05_advanced.ipynb")

In [ ]:
#!pip install --upgrade networkx

In [ ]:
from IPython.core.display import HTML
HTML("""
<style>
.imagesource {
    font-size: xx-small;
}
</style>
""")

from datascience import *

import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
plt.style.use('fivethirtyeight')

import os
import networkx as nx

from IPython.core.display import HTML
def css_styling():
    styles = open("custom_style.css", "r").read()
    return HTML(styles)
css_styling()

## Advanced Complete Network Data

You have had practice working with network data that includes node attributes, and exploring mathematical models for quantifying these networks in Lab 3 and Lab 4. This homework builds on your previous experience to work with complete network data.

As you may remember, the Add Health dataset we have been working with stores information about network connections in an edge list. The Add Health dataset also has information about the nodes; it has their grade, their gender, and their race/ethnicity.


# Question 1: 
Load the functions that allow you to read in the Add Health dataset.

_Hint: review the code in Lab 3_

In [ ]:

def read_add_health_edges(network_id, path=data_path):
    ...
    return(network.to_undirected())


def read_add_health_attributes_oneperrow(network_id, net, path=data_path):
    ...
    return(net)


def read_add_health_attributes(network_id, net, path=data_path):
    ...
    return(net)


def read_add_health_network(network_id):
   ...
    return(this_net)

In [ ]:
grader.check("q1")

# Question 2:

Use the read_add_health_network function to load the datasets into memory. Remember to omit networks 1 and 48.

_Hint: again, review the code in Lab 3_

In [ ]:
add_health_ids = ...
add_health_networks = ...

In [ ]:
grader.check("q2")

# Question 3:

Fill in the definition of a helper function to calculate the average degree of the given network.

_Hint: remember that the average degree is 2 times the number of edges, divided by the number of nodes_

In [ ]:

def average_degree(net):
    ## Helper function to calculate the average degree of a network
    
    ##Parameters
    ##----------
    ##net : nx.Graph
    ##    - The graph to calculate average degree
        
    ##Return
    ##------
    ##float
    ##    - Average degree of the network
    
    
    return(...)

avg_d=average_degree(add_health_networks[0])
avg_d

In [ ]:
grader.check("q3")

# Question 4

Write a loop that goes through each of the 84 Add Health networks and calculates the clustering coefficient and the number of nodes in the network. *(Please use the average clustering coefficient, implemented by the `average_clustering` function from the networkx package.)*

Store the results in a Table called `add_health_clustering` using columns called `num_nodes` and `avg_clustering_coef`.


In [ ]:
clustering = make_array()
num_nodes = make_array()

for g in ...:
    clustering = np.append(clustering, ...)
    num_nodes = np.append(num_nodes, ...)

add_health_clustering = Table().with_columns(['num_nodes', num_nodes,
                                              'avg_clustering_coef', clustering])
add_health_clustering

In [ ]:
grader.check("q4")

##  Average path length of biggest component

Remember that it really only makes sense to think about the average path length between two nodes that are in the same component. (Nodes in different components have no path between them.) Since some of the Add Health networks have more than one component, we'll start by picking out only the largest component in each network.

In [ ]:
# This might take a few seconds...
def get_biggest_component(network):
    """ Returns the largest connected component of a network
    
    Parameters
    ----------
    network : nx.Graph
        - A single networkx graph
    Returns
    -------
    nx.Graph
        - The largest component of the input network
    """
    components = [network.subgraph(c) for c in nx.connected_components(network)]
    biggest = max(components, key=len)
    return(biggest)

add_health_biggest_components = [get_biggest_component(g) for g in add_health_networks]

# Question 5:

The following code includes a loop that goes through the object created above (the largest component of each of the 84 Add Health networks) and calculates the average shortest path length and the number of nodes in the network. **This may take 3-5 minutes to run.**

Store the results in a Table called `add_health_sp` using columns called `num_nodes`, `avg_shortest_path`, and `avg_degree`.


In [ ]:
## NOTE: your code might take a little while
##       (~3-5 minutes) to run

avg_shortest_path = make_array()
num_nodes = make_array()
avg_degree = make_array()

for c in add_health_biggest_components:
    avg_shortest_path = np.append(avg_shortest_path, nx.average_shortest_path_length(c))
    num_nodes = np.append(num_nodes, c.number_of_nodes())
    avg_degree = np.append(avg_degree, average_degree(c))


add_health_sp = ...
add_health_sp

In [ ]:
grader.check("q5")


### Exploring network attributes

We'll start by looking at the fraction of students in one school that is male.

Looking at the [dataset information](http://moreno.ss.uci.edu/data.html#adhealth), you can see that the Add Health sex variable has the values 1=male, 2=female, and 0=unreported.

**Practice** Use the `get_node_attributes` function to grab the sexes of the students in the first school by filling in the code below.

In [ ]:
net = add_health_networks[0]
sexes = nx.get_node_attributes(net, 'sex')
sexes

# Question 6:

Write another loop that calculates the assortativity coefficient for sex in each community. Store your results in an array called `add_health_r`.

In [ ]:
add_health_r = make_array()

for net in add_health_networks:
    net_r = ...
    add_health_r = ...
    
add_health_r

In [ ]:
grader.check("q6")

# Question 7:

Now let's take a deeper dive, focusin on a specific Add Health network:

In [ ]:
first_add_health = add_health_networks[0]


Calculate the assortativity coefficient for sex in `first_add_health`.

In [ ]:
q7 = ...
q7

In [ ]:
grader.check("q7")


## Null Models and Homophily

We're interested in understanding whether or not there is homophily according to sex in this specific network. The assortativity coefficient is positive, which suggests that there is evidence in favor of homophily. However, we're in a similar situation to the example we saw in lecture: it seems possible that, actually, there is no homophily by sex in this network; rather, the network is assembled as the result of a random process and, just by chance, we happened to get a network that had a positive assortativity coefficient.

To assess how likely this possibility is, we will set up a *null model* and to compare what we see in the real world to what we would see if the null model were true. Remember that a null model describes the world in the absence of the phenomenon we are interested in; here, it describes a world in which networks are formed without any homophily by sex. The first_add_health network would be the observed network.

For this null model, we will assume that the network structure is fixed, but that the gender of each node is randomly assigned. (Another way of saying this is that we *condition on* the observed network structure.)

We'd like to know what the distribution of assortativity coefficients would look like under this null model. Then we can see how likely (or unlikely) the observed assortativity coefficient would be if the null model were true.

In [ ]:
import random

def shuffle_attribute(net, att): # the two inputs are: the network, and the attribute we work with
    att_dict = nx.get_node_attributes(net, att) # get the dictionary of the network
    
    # we want a new copy of the network (we don't want to clobber the original one)
    newnet = net.copy()
    
    # create a dictionary mapping node id to shuffled attribute values
    node_ids = att_dict.keys()
    att_vals = list(att_dict.values())
    random.shuffle(att_vals)
    
    new_att = dict(zip(node_ids, att_vals)) # create the new dictionary
    
    # assign the newly shuffled attribute values
    nx.set_node_attributes(newnet, name=att, values=new_att)
    
    return(newnet)

If you run the following cell a few times, you should see that the sex value is getting shuffled

In [ ]:
test = shuffle_attribute(first_add_health, 'sex')
test.nodes[1]['sex']

# Question 8:

Fill in the code below to reshuffle sex in the `first_add_health` network 1000 times. Record the assortativity coefficient for each reshuffled network.

In [ ]:
# Hint: use the correct index
first_add_health = add_health_networks[...]

null_network_r = make_array()

for _ in range(...):
    # shuffle the sex for the current network
    cur_net = shuffle_attribute(first_add_health, ...)
    # record the coefficient of the shuffled current network by appending it to the array
    null_network_r = np.append(null_network_r, nx.attribute_assortativity_coefficient(..., ...))

In [ ]:
grader.check("q8")

<!-- END QUESTION -->

Let's make a histogram to be able to visualize the 1000 iterations.

In [ ]:
Table().with_column('null_network_r', null_network_r).hist()

# Question 9:

Where is the center of the distribution for our null model?

_type your response here_

# Question 10: 

Now calculate the assortativity coefficient for the observed network (the first Add Health network) that you based the random networks on. Call this `observed_r`.

In [ ]:
observed_r = ...
observed_r

In [ ]:
grader.check("q10")

# Question 11:

Where in the null distribution does the observed value fall? What does this suggest about how likely the observed value is to have been generated from the null model?

---

To double-check your work, the cell below will rerun all of the autograder tests.

In [ ]:
grader.check_all()

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Please upload the .zip file to Gradescope.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(run_tests=True)